Gaby Branin

Making It (ENAS 400)

# Calendar Break Scheduler

In [1]:
import datetime as dt
import pandas as pd
import numpy as np

### fake schedule
contains busy times, beginning of day, and end of day.

In [2]:
start1 = dt.time(9, 0, 0)
end1 = dt.time(10, 0, 0)
start2 = dt.time(11, 30, 0)
end2 = dt.time(12, 0, 0)
start3 = dt.time(14, 0, 0)
end3 = dt.time(14, 15, 0)
start4 = dt.time(14, 30, 0)
end4 = dt.time(16, 0, 0)

BOD = dt.time(8, 0, 0)
EOD = dt.time(17, 0, 0)

In [3]:
busyTimes = pd.DataFrame({'start': [start1, start2, start3, start4], 'end': [end1, end2, end3, end4]})

print("User BUSY Times")
display(busyTimes)

User BUSY Times


,start,end
0,09:00:00,10:00:00
1,11:30:00,12:00:00
2,14:00:00,14:15:00
3,14:30:00,16:00:00


## find free times

In [4]:
def getFreeTimes(busy_times, user_BOD, user_EOD):  
    
#busy_times: comes from user calendar
#user_BOD: user beginning of day (input)
#user_EOD: user end of day (input)
    
    freeTimes = pd.DataFrame()
    
    #beginning of day
    if busyTimes.iloc[0,0] > user_BOD:
        startFree_BOD = user_BOD
        end_first_free = busy_times.iloc[0, 0]

        first_free = pd.DataFrame({'start': [startFree_BOD] , 'end': [end_first_free]})

        freeTimes = freeTimes.append(first_free)
        
    
    #during day events
    for i in np.arange(0, len(busy_times)-1):

        startFree = busy_times.iloc[i, 1]
        endFree = busy_times.iloc[i+1, 0]

        free = pd.DataFrame({'start': [startFree] , 'end': [endFree]})

        freeTimes = freeTimes.append(free)
    
    #end of day case
    if busy_times.iloc[len(busy_times)-1, 1] < user_EOD:
        startFree_EOD = busy_times.iloc[len(busy_times)-1, 1]
        EOD = user_EOD
        
        last_free = pd.DataFrame({'start': [startFree_EOD] , 'end': [EOD]})
        freeTimes = freeTimes.append(last_free)
    
    return freeTimes

In [5]:
user_freeTimes = getFreeTimes(busyTimes, BOD, EOD)

print("User FREE times")
display(user_freeTimes)

User FREE times


,start,end
0,08:00:00,09:00:00
0,10:00:00,11:30:00
0,12:00:00,14:00:00
0,14:15:00,14:30:00
0,16:00:00,17:00:00


## schedule breaks

In [6]:
#adds specific increment of time to a time value 
#(witout having to use .timedelta, which would cause me to switch all the .time() to .datetime())

def add_secs_to_time(time_val, secs_to_add):
    secs = (time_val.hour * 3600) + (time_val.minute * 60) + (time_val.second)
    secs += secs_to_add

    return dt.time(secs // 3600, (secs % 3600) // 60, secs % 60)

In [7]:
def find_breakTimes(freeTimes, break_length, time_between_breaks, time_after_busy, user_BOD):

    breakTimes = pd.DataFrame()

    #first break of the day:
    startBreak = add_secs_to_time(user_BOD, 45*60)
    endBreak = add_secs_to_time(startBreak, break_length*60)
    one_break = pd.DataFrame({'start': [startBreak] , 'end': [endBreak]})

    breakTimes = breakTimes.append(one_break)


    for i in np.arange(0,len(freeTimes)):
        #break starts user input amount of mins after beginning of free time (= end of busy time)
        startBreak = add_secs_to_time((freeTimes.iloc[i, 0]), time_after_busy*60)

        #currently a 10 minute break -- should be adjustable for user input
        endBreak = add_secs_to_time(startBreak, break_length*60)

        possible_break = pd.DataFrame({'start': [startBreak] , 'end': [endBreak]})

        #schedule the break if the start and the end of the break are during free time
        #possible break start must be after already scheduled break end
        
        if possible_break.iloc[0,0] > breakTimes.iloc[0,1]: #for the first case
            if (freeTimes.iloc[i, 0]) < startBreak < (freeTimes.iloc[i, 1]):
                if (freeTimes.iloc[i, 0]) < endBreak < (freeTimes.iloc[i, 1]):
                    breakTimes = breakTimes.append(possible_break)

        #while the end of the break is less than the end of the free time, schedule more breaks
        while (possible_break.iloc[0,1] < freeTimes.iloc[i,1]):
            startBreak = add_secs_to_time((possible_break.iloc[0,1]), time_between_breaks*60)

            #currently a 10 minute break -- should be adjustable for user input
            endBreak = add_secs_to_time(startBreak, break_length*60)

            possible_break = pd.DataFrame({'start': [startBreak] , 'end': [endBreak]})

            #schedule break if start and end are in free period
            if (freeTimes.iloc[i, 0]) < startBreak < (freeTimes.iloc[i, 1]):
                if (freeTimes.iloc[i, 0]) < endBreak < (freeTimes.iloc[i, 1]):
                    breakTimes = breakTimes.append(possible_break)
                    
    return breakTimes


In [8]:
user_breakTimes = find_breakTimes(user_freeTimes, break_length=10, time_between_breaks=45, time_after_busy=15, user_BOD = BOD)
display(user_breakTimes)

,start,end
0,08:45:00,08:55:00
0,10:15:00,10:25:00
0,11:10:00,11:20:00
0,12:15:00,12:25:00
0,13:10:00,13:20:00
0,16:15:00,16:25:00
